https://crypto.stackexchange.com/questions/2249/how-does-one-attack-a-two-time-pad-i-e-one-time-pad-with-key-reuse

#### WARNING: DO NOT DO INVENT NOR IMPLEMENT YOUR OWN CRYPTO
This is for demonstration purposes and OTP attack game only!

In [1]:
import secrets

In [2]:
def gen_key(length):
    """Vrne nakljucni niz bajtov podane dolzine"""
    return secrets.token_bytes(length)

In [3]:
def xor_bytes(s1, s2):
    """Izvede operacijo XOR med podanima seznamoma bajtov in vrne seznam bajtov"""
    return bytes(a ^ b for a, b in zip(s1, s2))

In [4]:
assert xor_bytes([1, 2, 3], [3, 2, 1]) == bytes([2, 0, 2])
assert xor_bytes([0, 0, 0], [3, 2, 1]) == bytes([3, 2, 1])
assert xor_bytes([1, 1, 1], [1, 1, 1]) == bytes([0, 0, 0])

In [5]:
def enc_otp(key, pt):
    """Sifrirni algoritem enkratne podloge"""
    return xor_bytes(key, pt)

In [6]:
assert enc_otp([1, 2, 3], [1, 1, 1]) == xor_bytes([1, 2, 3], [1, 1, 1])

In [7]:
def dec_otp(key, ct):
    """Desifrirni algoritem enkratne podloge"""
    return xor_bytes(key, ct)

In [8]:
assert dec_otp([1, 2, 3], [1, 1, 1]) == xor_bytes([1, 2, 3], [1, 1, 1])

In [9]:
def dec_otp_partial(key, ct):
    """Desifrira samo tiste znake, kjer kljuc ni 0 -- ce je, kot znak nastavi *"""
    return bytes(k ^ c if k > 0 else ord("*")  for k, c in zip(key, ct))

In [10]:
assert dec_otp_partial([1, 0, 1], [2, 2, 2]) == bytes([3, ord("*"), 3])
assert dec_otp_partial([1, 1, 1], [2, 2, 2]) == dec_otp([1, 1, 1], [2, 2, 2])

In [11]:
def prepare_pt():
    def normalize(msg):
        msg = msg.lower()
        msg = msg.replace("č", "c")
        msg = msg.replace("š", "s")
        msg = msg.replace("ž", "z")
        msg = " ".join(msg.split())
        return "".join(x for x in msg if x in 'abcdefghijklmnoprstuvz ')

    data = [
    "Slovenija je evropska država z zemljepisno lego na skrajnem severu Sredozemlja in na skrajnem jugu Srednje Evrope",
    "Skozi slovensko zgodovino so pomembni kulturni vplivi prihajali iz srednjeevropskega in apeninskega kulturnega prostora.",
    "Slovenija ima gospodarsko ureditev, ki temelji na prostem trgu, zdaj je po mednarodnih merilih uvrščena med države z zelo visokim indeksom človekovega razvoja",
    "Poimenovanje za Slovence izvira iz besede Slovani izvor te pa je nejasen je izpeljanka iz korena slov ki je med drugim ohranjen v nekaterih vodnih imenih širom vzhodne Evrope",
    "Prvič je bilo ime uporabljeno za državno tvorbo Federalna Slovenija, ki je po sklepu zasedanja Slovenskega narodnoosvobodilnega sveta pristopila v Demokratično federativno Jugoslavijo",
    "Slovanski predniki današnjih Slovencev so se na ilirsko ozemlje Slovenije naselili v sestem stoletju. V sedmem stoletju se je oblikovala Karantanija, prva država alpskih Slovanov.",
    "Okoli leta tisoc so bili napisani Brižinski spomeniki, prvi pisni dokument v slovenščini in prvi slovanski zapis v latinici.",
    "V Pomladi narodov so med številnimi narodi tudi Slovenci s političnim programom zahtevali Zedinjeno Slovenijo",
    "Geografsko leži Slovenija v srednji Evropi oziroma v jugovzhodni Evropi na stičišču Alp, Dinarskega gorstva, Panonske nižine in Sredozemlja.",
    "Največ slovenskega površja sestavljajo kamnine iz časa, mezozoika (apnenec, dolomit) in kenozoika, torej terciarja (lapor, peščenjak, fliš) in kvartarja (prod, pesek, glina, konglomerat).",
    #"Tole je pa sedaj 1, 2, 3, 4 zelo zanimivo. Tole je pa sedaj 1, 2, 3, 4 zelo zanimivo. Tole je pa sedaj 1, 2, 3, 4 zelo zanimivo. zelo zanimivo. zelo zanimivo."
    ]
    # target = "ce sifriras z enkratno podlogo, ne uporabi istega kljuce vec kot enkrat"
    length = len(min(data, key=lambda x:len(x)))
    return [normalize(s)[:length] for s in data]

In [12]:
plain_texts = prepare_pt()
secret_key = gen_key(len(max(plain_texts, key=lambda x:len(x))))
cipher_texts = [enc_otp(secret_key, pt.encode("utf-8")) for pt in plain_texts]

### Nekaj namigov
- Kodiranje je ASCII. V abecedi ni šumnikov, števk in ločilČ imamo le (male tiskane) črke in presledek
- Pri črki bo bit 6 vedno nastavljen. To pomeni, da je vsaka črka zakodirana z znakom, ki je > 64
- Pri presledku je bit 6 vedno nenastavljen. Presledek in ASCII vrednost 32
- Ko naredimo XOR črke in presledka bo bit 6 nastavljen. Torej če je ASCII vrednost v $d$ nad 64, to pomeni, da gledamo vrednost, ki je nastala z XOR presledka in neke črke: Če XOR-amo 32 (presledek) dobimo črko v enem čistopisu (sicer ne vemo katerem)


| $\oplus$       | A-Z (65-90) | presledek (32) |
| ---------------|-------------|----------------|
| A-Z (65-90)    |    $\leq 32$  |    $\geq 65$     |
| Presledek (32) |    $\geq 65$  |       $0$      |

In [13]:
def multiple_xor1(pos, cts):
    for c1 in cts:
        if all(c1[pos] ^ c2[pos] >= 65 or c1[pos] ^ c2[pos] == 0 for c2 in cts if c2 != c1):
            return c1[pos] ^ ord(' ')
    
    return 0

def multiple_xor(pos, cts):
    # na dani poziciji znak tajnopisa paroma XORamo z istoleznim znakom v vsakem drugem tajnopisu
    # ce se za nek znak izkaze, da je rezultat te operacije vedno vrednost >= 65 ali == 0
    # potem je znak na dani poziciji presledek
    # znak XOR-amo s presledkom, dobimo kljuc in ga vrnemo
    # v nasprotnem primeru za znak kljuca vrnemo vrednost 0
    for c1 in cts:
        is_c1_space = True
        
        for c2 in cts:
            if c1 == c2:
                continue
            
            if 0 < c1[pos] ^ c2[pos] < 65:
                is_c1_space = False
            
        if is_c1_space:
            return c1[pos] ^ ord(' ')    

    return 0

def guess_key(ciphertexts):
    key = bytearray(len(ciphertexts[0]))

    for i in range(len(ciphertexts[0])):
        key[i] = multiple_xor1(i, ciphertexts)
    return key


gk = guess_key(cipher_texts)
print(sum(1 for a, g in zip(secret_key, gk) if a == g) / len(secret_key))
print(dec_otp_partial(gk, cipher_texts[0]).decode("ascii"))

0.7889908256880734
*l***ni*a j* e*ropska **zava z zemlj*pi*no lego na *krajnem *e*er* s*edozemlj* in na *kr*jnem *ug* sre**je e*


In [14]:
for c in cipher_texts:
    print(dec_otp_partial(gk, c).decode("ascii"))

*l***ni*a j* e*ropska **zava z zemlj*pi*no lego na *krajnem *e*er* s*edozemlj* in na *kr*jnem *ug* sre**je e*
*k*** s*ove*sk* zgodov**o so pomembn* k*lturni vpli*i prihaj*l* i* s*ednjeevr*pskega *n *penin*ke*a ku**urne*
*l***ni*a i*a *ospodar**o ureditev k* t*melji na pr*stem trg* *da* j* po medn*rodnih *er*lih u*rs*ena **d dr*
*o***no*anj* z* sloven** izvira iz b*se*e slovani i*vor te p* *e *ej*sen je i*peljank* i* kore*a *lov ** je *
*r*** j* bi*o *me upor**ljeno za drz*vn* tvorbo fed*ralna sl*v*ni*a *i je po *klepu z*se*anja *lo*ensk**a na*
*l***ns*i p*ed*iki dan**njih slovenc*v *o se na ili*sko ozem*j* s*ov*nije nas*lili v *es*em st*le*ju v**edme*
*k*** l*ta *is*c so bi** napisani br*zi*ski spomeni*i prvi p*s*i *ok*ment v s*ovensci*i *n prv* s*ovan**i za*
* ***la*i n*ro*ov so m** stevilnimi *ar*di tudi slo*enci s p*l*ti*ni* program*m zahte*al* zedi*je*o sl**enij*
*e***af*ko *ez* sloven**a v srednji *vr*pi oziroma * jugovzh*d*i *vr*pi na st*ciscu a*p *inars*eg* gor**va p*
*a***c *lo

In [15]:
gk[0] = cipher_texts[0][0] ^ ord("s")
gk[2] = cipher_texts[0][2] ^ ord("o")
gk[3] = cipher_texts[0][3] ^ ord("v")
gk[4] = cipher_texts[0][4] ^ ord("e")
gk[7] = cipher_texts[0][7] ^ ord("j")

### Gnetljivost tajnopisa

Ana želi poslati zaupno pošto Boru, vas, ki igrate vlogo napadalca Nandija, pa vsebina sporočila _zares_ zanima.

Na srečo vam gre nekaj reči na roko. 

Prvič, Anin računalnik nima internetne povezave, vaš mobilni telefon pa. Zato ji prijazno ponudite, da zanjo postavite mobilno dostopno točko, preko katere se bo lahko povezala v internet in dostopala do poštnega strežnika. Poštni strežnik bo nato sporočilo dostavil Boru. 

Ana in Bor uporabljata poseben protokol za pošto: protokol FMTP -- Funny Mail Transfer Protocol.  Gre za preprost besedilno-osnovan protokol: prva vrstica označuje naslov prejemnika, druga naslov pošiljatelja, tretja zadevo, četrta je vedno prazna in na koncu je sporočilo.

Vse, kar mora Anin poštni odjemalec storiti, da pošlje sporočilo, je, da strežniku FMTP dostavi besedilni niz, podoben naslednjemu.

```txt
prejemnik@enadomena.com
posiljatelj@drugadomena.com
Zadeva sporocila

<Samo sporocilo>
```

Vse morebitne predhodne ali zaključne presledke v vsaki vrstici odstranimo pred obdelavo. Na primer, e-pošto zgoraj bi lahko napisali kot spodnjo in ne bi bilo nobene razlike.


```txt
                    prejemnik@enadomena.com               
    posiljatelj@drugadomena.com                    
Zadeva sporocila

<Samo sporocilo>
```

Tretjič, Ana vam v dobri veri -- kakšna naivnost! -- pove, da pošilja elektronsko sporočilo Boru in da uporablja protokol FMTP. (Torej poznate strukturo sporočila in vsebino prve vrstice čistopisa.)

Četrtič, Ana uporablja tokovno šifro brez kakršnih koli storitev zagotavljanja celovitosti.

Ker nastopate v vlogi **posrednika**, lahko sedaj spremenite sporočilo tako, da strežnik FMTP ne bo poslal pošte Boru, temveč jo bo poslal na naslov, ki ga nadzirate vi, npr. `nandi@obvlada.si`.

In [16]:
# Ana in Bor si enkrat tedensko v zivo izmenjata 1000 bajtov
# nakljucnih vrednosti za morebitne potrebe sifriranja
ana_bor_psk = gen_key(1000)

In [17]:
# Ana pripravi sporočilo
email = """bor@student.uni-lj.si
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.""".strip()

# In ga šifrira z enkratno podlogo
ct = enc_otp(ana_bor_psk, email.encode("utf8"))

# Nato ga pošlje (to simuliramo) na streznik FMTP
print("Ana posilja sifrirano sporocilo na streznik")
print("Tajnopis (HEX):", ct.hex())

Ana posilja sifrirano sporocilo na streznik
Tajnopis (HEX): 711e890583461c995839d588aa79039ab2b1d865ceb84e871b1746b1cd17f39810c893d9942d84fc3606395287cbc4c1d2390a05b893de0286e8faa2831b0af076caeb8f1e5d8064fe32


Sedaj je na potezi napadalec Nandi.

Implementirajte funckijo `change_ct(ct, new_email)`, ki na vhodu prejme tajnopis in email naslov. Spremenite tajnopis tako, da ga bo strežnik FMTP še vedno brez dešifriral, a kot prejemnik ne bo naveden Bor, temveč email naslov, ki je podan v argumentu `new_email`.

In [18]:
def change_ct(ct, new_email):
    cur = "bor@student.uni-lj.si"
    new = "nandi@obvlada.si     "
    
    l = bytearray(ct)
    
    for i, (c, n) in enumerate(zip(cur, new)):
        l[i] = l[i] ^ ord(c) ^ ord(n)
        
    return l

Z poganjanjem spodnje celice lahko preverite, ali vaša funkcija deluje pravilno. 

In [19]:
def fmtp_receive(key, ct):
    print("Streznik FMTP prejel sporocilo:", ct.hex())
    pt = dec_otp(key, ct)
    print("Desifrirano sporocilo:")
    print(pt.decode("utf8"))

fmtp_receive(ana_bor_psk, change_ct(ct, "nandi@obvlada.si"))

Streznik FMTP prejel sporocilo: 7d1095219972069f4b3bc0c2be3919defefbd63687b84e871b1746b1cd17f39810c893d9942d84fc3606395287cbc4c1d2390a05b893de0286e8faa2831b0af076caeb8f1e5d8064fe32
Desifrirano sporocilo:
nandi@obvlada.si     
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.
